In [235]:
import keras
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Setting seed for reproducability
  

from sklearn import preprocessing, svm
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras import models
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline
from decimal import Decimal
import warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,Normalizer, MinMaxScaler

from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
import collections
import itertools

In [339]:
df=pd.read_csv('C:\\Users\\lengada1\\NCSU\\ten_skus.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.weekday_name

day_dummy=pd.get_dummies(df.Day)
df=pd.concat([df,day_dummy],axis=1)
df.drop(['Day','Date'],inplace=True,axis=1)


In [346]:
def split_df(df):
    d={}
    split_by_list=np.unique(df.id.values)
    for sku in split_by_list:
        d[sku]=df[df.id==sku]
    return d

df_dict=split_df(df);

In [347]:
y={}
for sku in list(df_dict.keys()):
    y[sku]=df_dict[sku]['Sales']

In [348]:
X={}
for sku in list(df_dict.keys()):
    X[sku]=df_dict[sku]
    X[sku]=X[sku].drop(['id','DayOfWeek','Customers','High_Var','Luxury','Sales'],axis=1)

In [349]:
for sku in list(df_dict.keys()):
    for obs in range(1,8):
        X[sku]["Sales_T"+str(obs)]=df_dict[sku]['Sales'].shift(obs)

In [350]:
for sku in list(df_dict.keys()):
    X[sku]["Mov_avg"]=pd.rolling_mean(df_dict[sku]['Sales'], window=7).shift(1) 

In [351]:
#Cut lagged vars NAs off top 
cut_lag=7;
for sku in list(df_dict.keys()):
    y[sku]=y[sku][cut_lag:]
    y[sku].reset_index(drop=True, inplace=True)
    
for sku in list(df_dict.keys()):
    X[sku]=X[sku][cut_lag:]
    X[sku].reset_index(drop=True, inplace=True)

In [352]:
X[1].head()

,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Sales_T1,Sales_T2,Sales_T3,Sales_T4,Sales_T5,Sales_T6,Sales_T7,Mov_avg
0,1,1,1,2013,1,0,0,0,0,0,1,0,7176.0,0.0,4997.0,4486.0,4327.0,5530.0,0.0,3788.000000
1,1,1,1,2013,1,0,0,0,0,0,0,1,5580.0,7176.0,0.0,4997.0,4486.0,4327.0,5530.0,4585.142857
2,1,1,1,2013,1,0,0,0,0,1,0,0,5471.0,5580.0,7176.0,0.0,4997.0,4486.0,4327.0,4576.714286
3,1,1,1,2013,1,1,0,0,0,0,0,0,4892.0,5471.0,5580.0,7176.0,0.0,4997.0,4486.0,4657.428571
4,1,0,0,2013,1,0,0,1,0,0,0,0,4881.0,4892.0,5471.0,5580.0,7176.0,0.0,4997.0,4713.857143


In [353]:
std={}
for sku in list(df_dict.keys()):
    std[sku] = preprocessing.StandardScaler().fit(X[sku])
    X[sku] = std[sku].transform(X[sku])


In [354]:
trials=2;
folds=4;
kf=KFold(n_splits=folds, shuffle=True, random_state=i)

In [355]:
RF=RandomForestRegressor();
# use a full grid over all parameters
p_grid = {
          "n_estimators": [800,1000,1200],
             # 'min_samples_leaf': [1, 2],
              "criterion": ['mae'],
              'max_features':['auto',13, 14,15]
         }
rf_models=[];
rf_params=[];
for sku in list(df_dict.keys()):
    #rf_models.append(cross_val_score(RF,X[sku],y[sku],cv=cv, scoring ='mean_absolute_error').mean().round(0) )
    for i in range(trials): 
        clf = GridSearchCV(estimator=RF, param_grid=p_grid, cv=kf)
        clf.fit(X[sku],y[sku])
        rf_models.append(clf.best_score_)
        rf_params.append(clf.best_params_)

In [356]:
rf_keys=[]
for i in range(len(rf_params)):
    rf_keys.append(  list(rf_params[i].values() )   )
counts = collections.defaultdict(int)
for collab in rf_keys:
    #collab.sort()
    for pair in itertools.combinations(collab,4):
        counts[pair] += 1

for pair, freq in counts.items():
    print(pair, freq)

In [258]:
GB=ensemble.GradientBoostingRegressor()

p_grid = {    'n_estimators': [ 1000],  #more = better from my results
              'max_depth': [10], # Lower is better?  10 was better than 20, 30, 40
              'loss':['ls'],
              'learning_rate':[.01, .05,.1],  #small was better. 0.01 better than 0.05 or 0.10
          'max_features':['auto',15,17]  #auto uses all features which isn't as strong, 15 > 17
         
         }
gb_models=[];
gb_params=[]
for sku in list(df_dict.keys()):
    #gb_models.append(cross_val_score(GB,X[sku],y[sku],cv=cv, scoring ='mean_absolute_error').mean().round(0)  )
    for i in range(trials):         
        kf = KFold(n_splits=4, shuffle=True, random_state=i)
        clf = GridSearchCV(estimator=GB, param_grid=p_grid, cv=kf)
        clf.fit(X[sku],y[sku])
        gb_models.append(clf.best_score_)
        gb_params.append(clf.best_params_)

In [261]:
gb_keys=[]
for i in range(len(gb_params)):
    gb_keys.append(  list(gb_params[i].values() )   )

    counts = collections.defaultdict(int)
for collab in gb_keys:
    #collab.sort()
    for pair in itertools.combinations(collab,5):
        counts[pair] += 1

for pair, freq in counts.items():
    print(pair, freq)

(0.01, 'ls', 10, 15, 1000) 4
(0.01, 'ls', 10, 15, 750) 5
(0.05, 'ls', 10, 15, 1000) 3
(0.05, 'ls', 30, 15, 1000) 2
(0.01, 'ls', 10, 15, 500) 7
(0.1, 'ls', 30, 15, 500) 1
(0.1, 'ls', 10, 15, 1000) 2
(0.05, 'ls', 10, 15, 500) 2
(0.05, 'ls', 10, 15, 750) 3
(0.1, 'ls', 10, 15, 500) 1


In [360]:
XG=xgb.XGBRegressor()

p_grid = {'learning_rate':[.01], #.01 beat out .05 and .1 above
          'n_estimators':[900, 1100], 
          'max_depth':[6,8,10],
          'gamma':[0,2],  #Gamma is regularization, higher = less parameters 
    

           'learning_rate':[.01, .05,.1]

           }


xg_models=[];
xg_params=[]
for sku in list(df_dict.keys()):
    
    for i in range(trials):         
        kf = KFold(n_splits=4, shuffle=True, random_state=i)
        clf = GridSearchCV(estimator=XG, param_grid=p_grid, cv=kf)
        clf.fit(X[sku],y[sku])
        xg_models.append(clf.best_score_)
        xg_params.append(clf.best_params_)

In [362]:
xg_keys=[]
for i in range(len(xg_params)):
    xg_keys.append(  list(xg_params[i].values() )   )

    counts = collections.defaultdict(int)
for collab in xg_keys:
    #collab.sort()
    for pair in itertools.combinations(collab,4):
        counts[pair] += 1

for pair, freq in counts.items():
    print(pair, freq)

(0, 0.01, 6, 900) 6
(2, 0.01, 6, 900) 11
(0, 0.1, 6, 1100) 1
(2, 0.1, 6, 900) 1
(2, 0.1, 6, 1100) 1


In [332]:
from keras import optimizers
adam1=keras.optimizers.Adam(lr=0.001)
adam2=keras.optimizers.Adam(lr=0.002)
adam3=keras.optimizers.Adam(lr=0.005)
)

rms1=keras.optimizers.rmsprop(lr=0.001)
rms2=keras.optimizers.rmsprop(lr=0.002)
rms3=keras.optimizers.rmsprop(lr=0.1)
rms4=keras.optimizers.rmsprop(lr=0.15)


SGD1=keras.optimizers.SGD(lr=0.001)
SGD2=keras.optimizers.SGD(lr=0.002)
SGD3=keras.optimizers.SGD(lr=0.1)
SGD4=keras.optimizers.SGD(lr=0.15)



def create_MLP(optimizer=adam1):
    mlp=models.Sequential()
    mlp.add(Dense(20, input_dim=X[1].shape[1], activation='relu'))
    mlp.add(Dense(20, activation='relu'))
    #mlp.add(Dense(30, activation='relu'))
    mlp.add(Dense(10, activation='relu'))
    mlp.add(Dense(1))   
    mlp.compile(loss='mae', metrics=['mae'], optimizer=optimizer)    
    return mlp

MLP = KerasRegressor(build_fn=create_MLP,
                               verbose=0)

In [334]:
mlp_models=[]
p_grid = {'epochs':[400 ],
         'batch_size':[30],
         'optimizer':[ adam1, adam2, adam3]
         }




mlp_models=[];
mlp_params=[]
for sku in list(df_dict.keys()):
    
    for i in range(trials):         
        kf = KFold(n_splits=4, shuffle=True, random_state=i)
        clf = GridSearchCV(estimator=MLP, param_grid=p_grid, cv=kf)
        clf.fit(X[sku],y[sku])
        mlp_models.append(clf.best_score_)
        mlp_params.append(clf.best_params_) 

In [335]:
def step_decay(epoch):
   initial_lrate = 0.1
   drop = 0.5
   epochs_drop = 10.0
   lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
   return lrate
lrate = LearningRateScheduler(step_decay)

NameError: name 'LearningRateScheduler' is not defined

In [357]:
mlp_models

[-361.29688162574155,
 -398.24448522883938,
 -589.76047141335232,
 -682.54352779286432,
 -352.93132389496992,
 -370.70839634859624,
 -685.96007661054477,
 -386.72510562019551,
 -490.90667953083221,
 -370.42627570004385]